<a href="https://colab.research.google.com/github/KorStats/ocr_receipt/blob/main/%EA%B3%BC%EC%A0%95/111.%20%EC%9B%90%EB%B3%B8G%EB%8D%B0%EC%9D%B4%ED%84%B0%20%EC%A4%91%EB%B6%84%EB%A5%98%20%EB%AA%A8%EB%8D%B8%20%ED%95%99%EC%8A%B5%ED%95%98%EA%B8%B0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## 구글드라이브에서 파일 읽어오기

In [1]:
!pip install mxnet
!pip install gluonnlp pandas tqdm
!pip install sentencepiece
!pip install transformers==3.0.2
!pip install torch

#깃허브에서 KoBERT 파일 로드
!pip install git+https://git@github.com/SKTBrain/KoBERT.git@master
import torch
from torch import nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import gluonnlp as nlp
import numpy as np
from tqdm import tqdm, tqdm_notebook
#kobert
from kobert.utils import get_tokenizer
from kobert.pytorch_kobert import get_pytorch_kobert_model

#transformers
from transformers import AdamW
from transformers.optimization import get_cosine_schedule_with_warmup
#GPU 사용
device = torch.device("cuda:0")
#BERT 모델, Vocabulary 불러오기
bertmodel, vocab = get_pytorch_kobert_model()

     |████████████████████████████████| 47.3 MB 1.1 MB/s 
  Attempting uninstall: graphviz
    Found existing installation: graphviz 0.10.1
    Uninstalling graphviz-0.10.1:
      Successfully uninstalled graphviz-0.10.1
     |████████████████████████████████| 344 kB 4.1 MB/s 
  Created wheel for gluonnlp: filename=gluonnlp-0.10.0-cp37-cp37m-linux_x86_64.whl size=595730 sha256=2ed6931e23268ab21e20e10306aec08c5e51cab552bf6dbadf336d12c11fe64d
  Stored in directory: /root/.cache/pip/wheels/be/b4/06/7f3fdfaf707e6b5e98b79c041e023acffbe395d78a527eae00
Successfully built gluonnlp
     |████████████████████████████████| 1.2 MB 4.1 MB/s 
     |████████████████████████████████| 769 kB 4.0 MB/s 
     |████████████████████████████████| 3.0 MB 58.4 MB/s 
     |████████████████████████████████| 895 kB 59.2 MB/s 
  Cloning https://****@github.com/SKTBrain/KoBERT.git (to revision master) to /tmp/pip-req-build-kjnepu_s
  Running command git clone -q 'https://****@github.com/SKTBrain/KoBERT.git' /tmp/pi

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
from google.colab import drive
drive.mount('/content/gdrive')
!ln -s /content/gdrive/My\ Drive/ /mydrive
!ls /mydrive

Mounted at /content/gdrive
 산업분류자동화    '답안 작성용 파일.csv'   P_inc_model.pt
'Colab Notebooks'  'My Drive'


In [4]:
!unzip /content/gdrive/'My Drive'/산업분류자동화/'모델 개발용 자료'.zip -d /content/dataset
!unzip /content/gdrive/'My Drive'/산업분류자동화/'실습용 자료'.zip -d /content/dataset

Archive:  /content/gdrive/My Drive/산업분류자동화/모델 개발용 자료.zip
  inflating: /content/dataset/2. ╕Ё╡и┐ы░│╣▀┐ы└┌╖с.csv  
Archive:  /content/gdrive/My Drive/산업분류자동화/실습용 자료.zip
1. 실습용자료.txt:  mismatching "local" filename (1. ьЛдьК╡ьЪйьЮРыгМ.txt),
         continuing with "central" filename version
  inflating: /content/dataset/1. 실습용자료.txt  


In [5]:
import pandas as pd
df=pd.read_csv('/content/dataset/1. 실습용자료.txt', sep = "|", encoding = "euc-kr")
ts=pd.read_csv('/content/gdrive/MyDrive/답안 작성용 파일.csv', encoding = "euc-kr")

In [6]:
df=df[df['digit_1']=='G']

In [7]:
df=df.reset_index(drop=True)

In [8]:
df['digit_2'].value_counts(0)

47    162202
46     76835
45      7435
Name: digit_2, dtype: int64

## 데이터 전처리

In [9]:
df.head(10)

,AI_id,digit_1,digit_2,digit_3,text_obj,text_mthd,text_deal
0,id_0000002,G,47,472,상점내에서,일반인을 대상으로,채소.과일판매
1,id_0000003,G,46,467,절단하여사업체에도매,공업용고무를가지고,합성고무도매
2,id_0000004,G,47,475,영업점에서,일반소비자에게,열쇠잠금장치
3,id_0000013,G,47,474,매장에서,일반인 대상으로,의류소매
4,id_0000015,G,47,472,상점내,일반인을 대상으로,여러곡물소매판매
5,id_0000019,G,47,473,매장에서,일반인에게,핸드폰소매
6,id_0000024,G,47,472,점포에서,일반인대상으로소매,건어물
7,id_0000031,G,47,477,사업장에서,소비자를 대상으로,가정용가스 소매
8,id_0000032,G,46,467,생지를 구입하여,가공업체가공의뢰하여 완제품수출,"폴리에스테르.원단,직물"
9,id_0000039,G,47,471,가게에서,고객에게,잡화판매


In [10]:
df['text']=df['text_obj'].map(str)+" "+df['text_mthd'].map(str)+" "+df['text_deal'].map(str)
i=[]
for sentence in df['text']:
    a=sentence.replace('nan', '')
    i.append(a)
df['text']=pd.DataFrame(i)
df=df.drop(['text_obj','text_mthd', 'text_deal'], axis=1) # 기존에 있던 세개의 text변수 삭제
df

,AI_id,digit_1,digit_2,digit_3,text
0,id_0000002,G,47,472,상점내에서 일반인을 대상으로 채소.과일판매
1,id_0000003,G,46,467,절단하여사업체에도매 공업용고무를가지고 합성고무도매
2,id_0000004,G,47,475,영업점에서 일반소비자에게 열쇠잠금장치
3,id_0000013,G,47,474,매장에서 일반인 대상으로 의류소매
4,id_0000015,G,47,472,상점내 일반인을 대상으로 여러곡물소매판매
...,...,...,...,...,...
246467,id_0999972,G,47,473,매장에서 일반인에게 핸드폰소매
246468,id_0999975,G,47,474,매장에서 일반소비자에게 골프의류
246469,id_0999977,G,46,467,"고철,폐지 등"
246470,id_0999992,G,46,463,"사업장에서 다른소매자에게 육류,냉동,떡"


In [11]:
# 결측값 확인
df.isnull().sum()

AI_id      0
digit_1    0
digit_2    0
digit_3    0
text       0
dtype: int64

In [12]:
df.shape

(246472, 5)

In [13]:
df['digit_2'].value_counts()

47    162202
46     76835
45      7435
Name: digit_2, dtype: int64



#### **추후 과제: 분류가 현재 매우 불균형함 / 정규화를 이용해서 균형을 맞추면 정확도가 올라가는지 확인 필요**


In [14]:
df.loc[(df['digit_2'] == 45), 'digit_2'] = 0  
df.loc[(df['digit_2'] == 46), 'digit_2'] = 1  
df.loc[(df['digit_2'] == 47), 'digit_2'] = 2

In [15]:
data_list = []
for q, label in zip(df['text'], df['digit_2'])  :
    data = []
    data.append(q)
    data.append(str(label))

    data_list.append(data)

print(data_list[:5])

[['상점내에서 일반인을 대상으로 채소.과일판매', '2'], ['절단하여사업체에도매 공업용고무를가지고 합성고무도매', '1'], ['영업점에서 일반소비자에게 열쇠잠금장치', '2'], ['매장에서 일반인 대상으로 의류소매', '2'], ['상점내 일반인을 대상으로 여러곡물소매판매', '2']]


## Train data & test data

In [16]:
from sklearn.model_selection import train_test_split             
dataset_train, dataset_test = train_test_split(data_list, test_size=0.1, random_state=0) #추후에 test_size 수정 필요

# KoBERT 입력 데이터로 만들기

In [17]:
class BERTDataset(Dataset):
    def __init__(self, dataset, sent_idx, label_idx, bert_tokenizer, max_len,
                 pad, pair):
        transform = nlp.data.BERTSentenceTransform(
            bert_tokenizer, max_seq_length=max_len, pad=pad, pair=pair)

        self.sentences = [transform([i[sent_idx]]) for i in dataset]
        self.labels = [np.int32(i[label_idx]) for i in dataset]

    def __getitem__(self, i):
        return (self.sentences[i] + (self.labels[i], ))

    def __len__(self):
        return (len(self.labels))

In [18]:
# Setting parameters
max_len = 64
batch_size = 64
warmup_ratio = 0.1
num_epochs = 5 #추후 num_epochs 증가할 필요 있음
max_grad_norm = 1
log_interval = 200
learning_rate =  5e-5

 ### 추후 과제 : 쓰이는 parameters 최적화 필요

In [19]:
#토큰화
tokenizer = get_tokenizer()
tok = nlp.data.BERTSPTokenizer(tokenizer, vocab, lower=False)

using cached model. /content/.cache/kobert_news_wiki_ko_cased-1087f8699e.spiece


In [20]:
data_train = BERTDataset(dataset_train, 0, 1, tok, max_len, True, False)
data_test = BERTDataset(dataset_test, 0, 1, tok, max_len, True, False)

In [21]:
data_train[0]

(array([   2, 1434, 7095, 6107,    3,    1,    1,    1,    1,    1,    1,
           1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
           1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
           1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
           1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
           1,    1,    1,    1,    1,    1,    1,    1,    1], dtype=int32),
 array(5, dtype=int32),
 array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       dtype=int32),
 2)

In [22]:
train_dataloader = torch.utils.data.DataLoader(data_train, batch_size=batch_size, num_workers=5)
test_dataloader = torch.utils.data.DataLoader(data_test, batch_size=batch_size, num_workers=5)

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 5 worker processes in total. Our suggested max number of worker in current system is 4, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


# KoBERT 학습 모델 만들기

In [23]:
class BERTClassifier(nn.Module):
    def __init__(self,
                 bert,
                 hidden_size = 768,
                 num_classes=3,   ##클래스 수 조정##
                 dr_rate=None,
                 params=None):
        super(BERTClassifier, self).__init__()
        self.bert = bert
        self.dr_rate = dr_rate
                 
        self.classifier = nn.Linear(hidden_size , num_classes)
        if dr_rate:
            self.dropout = nn.Dropout(p=dr_rate)
    
    def gen_attention_mask(self, token_ids, valid_length):
        attention_mask = torch.zeros_like(token_ids)
        for i, v in enumerate(valid_length):
            attention_mask[i][:v] = 1
        return attention_mask.float()

    def forward(self, token_ids, valid_length, segment_ids):
        attention_mask = self.gen_attention_mask(token_ids, valid_length)
        
        _, pooler = self.bert(input_ids = token_ids, token_type_ids = segment_ids.long(), attention_mask = attention_mask.float().to(token_ids.device))
        if self.dr_rate:
            out = self.dropout(pooler)
        return self.classifier(out)

In [24]:
#BERT 모델 불러오기
model = BERTClassifier(bertmodel,  dr_rate=0.5).to(device)

#optimizer와 schedule 설정
no_decay = ['bias', 'LayerNorm.weight']
optimizer_grouped_parameters = [
    {'params': [p for n, p in model.named_parameters() if not any(nd in n for nd in no_decay)], 'weight_decay': 0.01},
    {'params': [p for n, p in model.named_parameters() if any(nd in n for nd in no_decay)], 'weight_decay': 0.0}
]

optimizer = AdamW(optimizer_grouped_parameters, lr=learning_rate)
loss_fn = nn.CrossEntropyLoss()

t_total = len(train_dataloader) * num_epochs
warmup_step = int(t_total * warmup_ratio)

scheduler = get_cosine_schedule_with_warmup(optimizer, num_warmup_steps=warmup_step, num_training_steps=t_total)

#정확도 측정을 위한 함수 정의
def calc_accuracy(X,Y):
    max_vals, max_indices = torch.max(X, 1)
    train_acc = (max_indices == Y).sum().data.cpu().numpy()/max_indices.size()[0]
    return train_acc
    
train_dataloader

/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:309: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,


# KoBERT 모델 학습시키기

In [25]:
for e in range(num_epochs):
    train_acc = 0.0
    test_acc = 0.0
    model.train()
    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(train_dataloader)):
        optimizer.zero_grad()
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length= valid_length
        label = label.long().to(device)
        out = model(token_ids, valid_length, segment_ids)
        loss = loss_fn(out, label)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_grad_norm)
        optimizer.step()
        scheduler.step()  # Update learning rate schedule
        train_acc += calc_accuracy(out, label)
        if batch_id % log_interval == 0:
            print("epoch {} batch id {} loss {} train acc {}".format(e+1, batch_id+1, loss.data.cpu().numpy(), train_acc / (batch_id+1)))
    print("epoch {} train acc {}".format(e+1, train_acc / (batch_id+1)))
    
    model.eval()
    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(test_dataloader)):
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length= valid_length
        label = label.long().to(device)
        out = model(token_ids, valid_length, segment_ids)
        test_acc += calc_accuracy(out, label)
    print("epoch {} test acc {}".format(e+1, test_acc / (batch_id+1)))

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  """


  0%|          | 0/3466 [00:00<?, ?it/s]

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 5 worker processes in total. Our suggested max number of worker in current system is 4, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


epoch 1 batch id 1 loss 1.0958013534545898 train acc 0.3125
epoch 1 batch id 201 loss 0.30429962277412415 train acc 0.6003575870646766
epoch 1 batch id 401 loss 0.2957434058189392 train acc 0.7598192019950125
epoch 1 batch id 601 loss 0.14349102973937988 train acc 0.8241992512479202
epoch 1 batch id 801 loss 0.22339828312397003 train acc 0.8572097378277154
epoch 1 batch id 1001 loss 0.1448478251695633 train acc 0.8774506743256744
epoch 1 batch id 1201 loss 0.10221269726753235 train acc 0.8910283097418817
epoch 1 batch id 1401 loss 0.18194188177585602 train acc 0.9008074589578873
epoch 1 batch id 1601 loss 0.09841909259557724 train acc 0.9082702217364147
epoch 1 batch id 1801 loss 0.06270129233598709 train acc 0.91377186285397
epoch 1 batch id 2001 loss 0.10644350200891495 train acc 0.9179550849575212
epoch 1 batch id 2201 loss 0.1624862253665924 train acc 0.921782712403453
epoch 1 batch id 2401 loss 0.04312313348054886 train acc 0.9249921907538525
epoch 1 batch id 2601 loss 0.183942258

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:23: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`


...

epoch 4 test acc 0.9640544041450777


  0%|          | 0/3466 [00:00<?, ?it/s]

epoch 5 batch id 1 loss 0.0964871197938919 train acc 0.96875
epoch 5 batch id 201 loss 0.041707877069711685 train acc 0.9774564676616916
epoch 5 batch id 401 loss 0.05972853675484657 train acc 0.9772054239401496
epoch 5 batch id 601 loss 0.022031908854842186 train acc 0.9779014143094842
epoch 5 batch id 801 loss 0.23389428853988647 train acc 0.9777036516853933
epoch 5 batch id 1001 loss 0.045259028673172 train acc 0.9783653846153846
epoch 5 batch id 1201 loss 0.0726718157529831 train acc 0.9790539134054954
epoch 5 batch id 1401 loss 0.06950168311595917 train acc 0.9791666666666666
epoch 5 batch id 1601 loss 0.07142887264490128 train acc 0.9790462991880075
epoch 5 batch id 1801 loss 0.0459272526204586 train acc 0.978935313714603
epoch 5 batch id 2001 loss 0.016245076432824135 train acc 0.979026111944028
epoch 5 batch id 2201 loss 0.04023772478103638 train acc 0.9791004089050431
epoch 5 batch id 2401 loss 0.0061956653371453285 train acc 0.9791102665556018
epoch 5 batch id 2601 loss 0.141

  0%|          | 0/386 [00:00<?, ?it/s]

epoch 5 test acc 0.9628400259067358


In [26]:
#학습 모델 저장
torch.save(model, 'G_ori_jung_model.pt')